<a href="https://colab.research.google.com/github/gowtham-0017/Tree_Species_Classifications_final/blob/main/fixed_tree_species_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload archive.zip

In [ ]:
with zipfile.ZipFile("/content/archive.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/tree_species_dataset")

In [ ]:
print("Dataset extracted successfully.")

Dataset extracted successfully.


In [ ]:
repo_path = "/content/tree_species_dataset/Tree_Species_Dataset"
print("Using dataset path:", repo_path)




Using dataset path: /content/tree_species_dataset/Tree_Species_Dataset


In [ ]:
image_dims = []
for class_folder in os.listdir(repo_path):
    class_path = os.path.join(repo_path, class_folder)
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                with Image.open(img_path) as img:
                    image_dims.append(img.size)
            except:
                continue

In [ ]:
widths, heights = zip(*image_dims)
print(f"Min size: {min(widths)}x{min(heights)} | Max size: {max(widths)}x{max(heights)}")

Min size: 141x128 | Max size: 4000x4032


In [ ]:
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [ ]:
train_generator = datagen.flow_from_directory(
    repo_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 1280 images belonging to 31 classes.


In [ ]:
val_generator = datagen.flow_from_directory(
    repo_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 320 images belonging to 31 classes.


In [ ]:
base_model = EfficientNetB0(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), weights='imagenet')
base_model.trainable = False

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 31)             │         3,999 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,217,538 (16.09 MB)

 Trainable params: 167,967 (656.12 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
history = model.fit(train_generator, validation_data=val_generator, epochs=5)
model.save("tree_species_model.h5")

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 126s 3s/step - accuracy: 0.0642 - loss: 3.4552 - val_accuracy: 0.0938 - val_loss: 3.3888
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - accuracy: 0.1073 - loss: 3.3923 - val_accuracy: 0.0938 - val_loss: 3.3984
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 106s 3s/step - accuracy: 0.0751 - loss: 3.4133 - val_accuracy: 0.0938 - val_loss: 3.3762
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 109s 3s/step - accuracy: 0.0897 - loss: 3.3961 - val_accuracy: 0.0938 - val_loss: 3.3779
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 107s 3s/step - accuracy: 0.0816 - loss: 3.4056 - val_accuracy: 0.0938 - val_loss: 3.3774


In [ ]:
basic_cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
basic_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
basic_cnn.fit(train_generator, validation_data=val_generator, epochs=5)
basic_cnn.save("basic_cnn_tree_species.h5")

Epoch 1/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 98s 2s/step - accuracy: 0.0673 - loss: 17.8129 - val_accuracy: 0.1031 - val_loss: 3.2711
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.2385 - loss: 3.0047 - val_accuracy: 0.2406 - val_loss: 2.8638
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.5908 - loss: 1.8046 - val_accuracy: 0.3469 - val_loss: 2.6145
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.9300 - loss: 0.4986 - val_accuracy: 0.3281 - val_loss: 2.7989
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.9934 - loss: 0.1007 - val_accuracy: 0.3781 - val_loss: 2.8374


In [ ]:
improved_cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
improved_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
improved_cnn.fit(train_generator, validation_data=val_generator, epochs=5)
improved_cnn.save("improved_cnn_model.h5")

Epoch 1/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 208s 5s/step - accuracy: 0.0843 - loss: 49.9313 - val_accuracy: 0.0531 - val_loss: 13.9991
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 195s 5s/step - accuracy: 0.0751 - loss: 3.5097 - val_accuracy: 0.0938 - val_loss: 47.6275
Epoch 3/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 197s 5s/step - accuracy: 0.0929 - loss: 3.4327 - val_accuracy: 0.1063 - val_loss: 70.4785
Epoch 4/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 210s 5s/step - accuracy: 0.0951 - loss: 3.6114 - val_accuracy: 0.0844 - val_loss: 88.9381
Epoch 5/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 203s 5s/step - accuracy: 0.1025 - loss: 3.4253 - val_accuracy: 0.0875 - val_loss: 89.7118


In [ ]:
print("All models trained and saved successfully!")

All models trained and saved successfully!


In [ ]:
model.save("tree_species_model.h5")
print("✅ Model saved as tree_species_model.h5")


✅ Model saved as tree_species_model.h5
